### Motivation
* Since we do not have any available feature layer to utilize for Zip codes. We will try to create our own layer after picking the subgeographies from arcgis.
* Unfortunately we are only receiving 1000 zip5 for california, not sure if there is any limitation on number of subgeographies we could extract from arcgis

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer, FeatureSet
from arcgis.geoenrichment import enrich, Country
from feature_layer_utils import get_enrichment_variables
import itertools
import sys
sys.path.append('../')
from utils import get_config

In [2]:
app_id = get_config("arcgis","clientid")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=18c3ezbC4CiHMPkIMgyaaLAaHG6UtA&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
usa = Country.get('US')

In [4]:
zips = usa.subgeographies.states['California'].zip5

In [5]:
print(f"Total Zip codes in California: {len(zips)}")

Total Zip codes in California: 1000


In [6]:
type(list(zips.values())[0])

arcgis.geoenrichment.enrichment.NamedArea

In [7]:
test_zip =list(zips.values())[0]

In [8]:
test_zip.geometry.spatial_reference

{'wkid': 4326, 'latestWkid': 4326}

In [9]:
named_areas_list =  list(zips.values())

`named_area.__studyarea__['ids'][0]` gives us the zip code AKA calling __studyarea__ function on NamedArea object.

In [10]:
features = []
for named_area in named_areas_list:
    features.append({"attributes": {'state_fips':'06','state_abbr':'CA','zip_code':named_area.__studyarea__['ids'][0]}
                    ,"geometry": named_area.geometry})

In [11]:
feature_set = FeatureSet(features=features,
           geometry_type="Polygon",
           spatial_reference={'wkid': 4326, 'latestWkid': 4326},
           )

In [12]:
zips_sdf = feature_set.sdf
print(f"Shape: {zips_sdf.shape}")
zips_sdf.head(4)

Shape: (1000, 5)


,state_fips,state_abbr,zip_code,OBJECTID,SHAPE
0,06,CA,90001,1,"{""rings"": [[[-118.24328988033204, 33.989010029..."
1,06,CA,90002,2,"{""rings"": [[[-118.23113987955173, 33.959050019..."
2,06,CA,90003,3,"{""rings"": [[[-118.26406584044037, 33.988588649..."
3,06,CA,90004,4,"{""rings"": [[[-118.28405987997593, 34.083080009..."


In [14]:
california_zip_code_fl = zips_sdf.spatial.to_featurelayer(title='California Zip Codes 1.0',
                                                        gis=gis,
                                                        folder='nourish_gis',
                                                        tags=['California','ZipCode'],
                                                        service_name='California Zip Codes 1.0'
                                                    )
california_zip_code_fl

<Item title:"California Zip Codes 1.0" type:Feature Layer Collection owner:akale_UCSDOnline>